In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import matplotlib.pyplot as plt; 
from collections import Counter
from collections import OrderedDict
from nltk.stem import PorterStemmer, WordNetLemmatizer
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
import heapq
import collections
import datetime
import time
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "./data/"]).decode("utf8"))

all_the_news_2017.json
articles1.csv
articles2.csv
articles3.csv
english.stop.txt
id2word.json
word2id.json



In [2]:
dictionarySize = 3000
def downloadNews():
    news1 = pd.read_csv('./data/articles1.csv')
    news2 = pd.read_csv('./data/articles2.csv')
    news3 = pd.read_csv('./data/articles3.csv')
    news1.index = range(0,                               news1.shape[0])
    news2.index = range(news1.shape[0],                  news1.shape[0]+news2.shape[0])
    news3.index = range(news1.shape[0]+news2.shape[0],   news1.shape[0]+news2.shape[0]+news3.shape[0])
    news = pd.concat([news1, news2, news3])
    return news

def getWordList(someContent, tupleLength):
    t=tupleLength
    aList = []
    for i in range(0,tupleLength):
        aList.append(someContent[i:len(someContent)-(t-i)])
    trans = list(map(list, zip(*(aList))))
    return list(map(''.join,trans))

#Gets a dictionary of Agencies to a List of their articles and a list of the Agencies
def getAgenciesContent():
    news = downloadNews()
    Agencies = news.publication.unique()
    AgencyContent = {}
    for aAgency in Agencies:
        AgenRows = news.loc[news['publication'] == aAgency]
        AgencyContent[aAgency] = AgenRows['content']
    return AgencyContent

def getPredictionTuples(someContent, tupleLength):
    prediction = someContent[tupleLength:len(someContent)-1]
    wordList = getWordList(someContent, tupleLength)
    return tuple(map(tuple, zip(*[wordList, prediction])))
#returns tuples of words with their associated output

def getContentDictionary(someContent, tupleLength):
    otherChars = string.punctuation + ' ' + ''

    totalCount = Counter()
    bigWordList = []
    for aArticle in someContent:
        aSplitList = list(re.split('(\W)', aArticle.lower()))
        aCleanSplitList = [chunk for chunk in aSplitList if chunk not in otherChars]
        wordList = getWordList(aCleanSplitList, tupleLength)
#         print('type of wordList is ' + type(wordList).__name__)
#         print('type of wordList[0] is ' + type(wordList[0]).__name__)
        bigWordList.extend(wordList)
        aCount = Counter(wordList)
        totalCount = aCount + totalCount
    return totalCount, bigWordList
#Returns a dictionary of words to their count in all content given

def getSorted(aList):
    def getCount(item):
        return -item[1]
    return sorted(aList, key=getCount)
#Sorts diction of words->Counts by the count value returns a list

def plotWordDict(tempDict):
    tempDict = dict(tempDict)
    objects = tempDict.keys()
    y_pos = np.arange(len(objects))
    performance = tempDict.values()

    plt.bar(y_pos, performance, align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('%')
    plt.title('Word Frequency')

    plt.show()
    
def getEmbedding(wordCol, aWord):
    hotEncod = np.full((1, len(wordCol)), 0)
    if aWord in wordCol:
        index = wordCol.index(aWord)
        hotEncod[0][index] = 1
    else: 
        hotEncod[0][len(wordCol)-1] = 1
    return hotEncod

def getEmbeddingIndex(wordCol, aWord):
    if aWord in wordCol:
        return wordCol.index(aWord)
    else: 
        return len(wordCol) - 1

def getWordFromEmbeding(wordCol, aWordVector):
    return getWordFromIndex(wordCol, np.nonzero(aWordVector == 1)[1][0])

def getWordFromIndex(wordCol, wordIndex):
    if wordIndex < len(wordCol)-1:
        outWord = wordCol[wordIndex]
    else: 
        outWord = './RareWord'
    return outWord
print('ran')

ran


In [3]:
news = downloadNews()
print(news.shape)
news

(142570, 10)


,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."
5,5,17288,"Sick With a Cold, Queen Elizabeth Misses New Y...",New York Times,Sewell Chan,2017-01-02,2017.0,1.0,NaN,"LONDON — Queen Elizabeth II, who has been b..."
6,6,17289,Taiwan’s President Accuses China of Renewed In...,New York Times,Javier C. Hernández,2017-01-02,2017.0,1.0,NaN,BEIJING — President Tsai of Taiwan sharpl...
7,7,17290,"After ‘The Biggest Loser,’ Their Bodies Fought...",New York Times,Gina Kolata,2017-02-08,2017.0,2.0,NaN,"Danny Cahill stood, slightly dazed, in a blizz..."
8,8,17291,"First, a Mixtape. Then a Romance. - The New Yo...",New York Times,Katherine Rosman,2016-12-31,2016.0,12.0,NaN,"Just how is Hillary Kerr, the founder of ..."
9,9,17292,Calling on Angels While Enduring the Trials of...,New York Times,Andy Newman,2016-12-31,2016.0,12.0,NaN,Angels are everywhere in the Muñiz family’s ap...


In [4]:
with open('english.stop.txt', 'r') as f:
    english_stop = f.read().splitlines()
english_stop = set(english_stop)

# manually add some specific stopwords
english_stop.add('ms')
english_stop.add('mr')

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def text_preprocess(text):
    otherChars = string.punctuation + ' ' + ''
    printable = set(string.printable)
    text = filter(lambda x: x in printable, text)
    token_list = list(re.split('(\W)', text.lower()))
    ps = PorterStemmer()
    #lemmatizer = WordNetLemmatizer()
    return [ps.stem(token) for token in token_list if (token not in otherChars) and \
            (token not in english_stop) and (not hasNumbers(token))]

In [5]:
vocabulary = set()
dates = []
texts = []
for i in xrange(news.shape[0]):
    text = text_preprocess(news.loc[i,:]['content'])
    date = news.loc[i,:]['date']
    vocabulary = vocabulary | set(text)
    texts.append(text)
    dates.append(date)
    if(i+1) % 1000 == 0:
        print('finish handling document %d'% i)

finish handling document 999
finish handling document 1999
finish handling document 2999
finish handling document 3999
finish handling document 4999
finish handling document 5999
finish handling document 6999
finish handling document 7999
finish handling document 8999
finish handling document 9999
finish handling document 10999
finish handling document 11999
finish handling document 12999
finish handling document 13999
finish handling document 14999
finish handling document 15999
finish handling document 16999
finish handling document 17999
finish handling document 18999
finish handling document 19999
finish handling document 20999
finish handling document 21999
finish handling document 22999
finish handling document 23999
finish handling document 24999
finish handling document 25999
finish handling document 26999
finish handling document 27999
finish handling document 28999
finish handling document 29999
finish handling document 30999
finish handling document 31999
finish handling doc

In [6]:
print(len(vocabulary))

234559


In [7]:
# clear news for memory management
import gc
news = None
gc.collect()

1471

In [8]:
# get frequent word from texts
cnt = Counter(word for text in texts for word in set(text))

In [9]:
# shrinking vocabulary
cnt = Counter({k:v for k, v in cnt.items() if v > 5 and v < 35256})

In [10]:
# build word2id, id2word
word2id = {}
id2word = {}
for idx ,(word, count) in enumerate(cnt.iteritems()):
    word2id[word] = idx
    id2word[idx] = word

In [12]:
# save word2id and id2word
import json

with open('./data/word2id.json','w') as f:
    json.dump(word2id, f)
    
with open('./data/id2word.json','w') as f:
    json.dump(id2word, f)

In [214]:
# change text into bag of words word2count
news_items = []
for i, text in enumerate(texts):
    id2count_per_doc = {}
    text_cnt = Counter(text)
    words = text_cnt.keys()
    for word in words:
        try:
            word_id = word2id[word]
            id2count_per_doc[word_id] = text_cnt[word]
        except KeyError:
            continue
    id2count_per_doc = id2count_per_doc.items()
    try:
        word_ids, counts = zip(*id2count_per_doc)
        word_distribution = (word_ids, counts)
        word_count = sum(counts)
        date = dates[i]
        if word_count > 50:
            news_items.append( [date, word_distribution, word_count] )
    except ValueError:
        pass

In [215]:
len(news_items)

136930

In [270]:
def try_parsing_date(text):
    for fmt in ("%Y-%m-%d", "%Y/%m/%d"):
        try:
            return datetime.datetime.strptime(text, fmt).date()
        except ValueError:
            pass
    raise ValueError('no valid date format found')

In [277]:
dated_news_item = []
for i, news_item in enumerate(news_items):
    try:
        datestr = news_item[0]
        word_distribution = news_item[1]
        word_count = news_item[2]
        date = try_parsing_date(datestr)
        dated_news_item.append([date, word_distribution, word_count])
    except TypeError:
        print(i, datestr) # filter data that do not have date

(58041, nan)
(58042, nan)
(58043, nan)
(58044, nan)
(58046, nan)
(58047, nan)
(58048, nan)
(58049, nan)
(58050, nan)
(58051, nan)
(58052, nan)
(58054, nan)
(58055, nan)
(58056, nan)
(58057, nan)
(58058, nan)
(58059, nan)
(58060, nan)
(58061, nan)
(58062, nan)
(58063, nan)
(58064, nan)
(58069, nan)
(58070, nan)
(58071, nan)
(58072, nan)
(58073, nan)
(58074, nan)
(58075, nan)
(58076, nan)
(58077, nan)
(58079, nan)
(58080, nan)
(58081, nan)
(58082, nan)
(58083, nan)
(58084, nan)
(58085, nan)
(58086, nan)
(58087, nan)
(58088, nan)
(58089, nan)
(58090, nan)
(58091, nan)
(58093, nan)
(58094, nan)
(58095, nan)
(58096, nan)
(58098, nan)
(58099, nan)
(58100, nan)
(58101, nan)
(58102, nan)
(58103, nan)
(58104, nan)
(58105, nan)
(58106, nan)
(58107, nan)
(58108, nan)
(58109, nan)
(58110, nan)
(58111, nan)
(58112, nan)
(58113, nan)
(58114, nan)
(58115, nan)
(58116, nan)
(58117, nan)
(58118, nan)
(58119, nan)
(58120, nan)
(58121, nan)
(58122, nan)
(58123, nan)
(58124, nan)
(58125, nan)
(58126, nan)

(59038, nan)
(59039, nan)
(59040, nan)
(59041, nan)
(59042, nan)
(59043, nan)
(59044, nan)
(59045, nan)
(59046, nan)
(59047, nan)
(59048, nan)
(59049, nan)
(59050, nan)
(59051, nan)
(59052, nan)
(59053, nan)
(59054, nan)
(59055, nan)
(59056, nan)
(59057, nan)
(59058, nan)
(59059, nan)
(59060, nan)
(59062, nan)
(59063, nan)
(59064, nan)
(59065, nan)
(59066, nan)
(59067, nan)
(59068, nan)
(59069, nan)
(59070, nan)
(59071, nan)
(59072, nan)
(59073, nan)
(59074, nan)
(59075, nan)
(59076, nan)
(59077, nan)
(59078, nan)
(59079, nan)
(59080, nan)
(59081, nan)
(59082, nan)
(59083, nan)
(59084, nan)
(59085, nan)
(59086, nan)
(59087, nan)
(59088, nan)
(59089, nan)
(59090, nan)
(59091, nan)
(59092, nan)
(59093, nan)
(59094, nan)
(59095, nan)
(59096, nan)
(59097, nan)
(59098, nan)
(59099, nan)
(59100, nan)
(59101, nan)
(59102, nan)
(59103, nan)
(59104, nan)
(59105, nan)
(59106, nan)
(59107, nan)
(59108, nan)
(59109, nan)
(59110, nan)
(59111, nan)
(59112, nan)
(59113, nan)
(59114, nan)
(59115, nan)

(59701, nan)
(59702, nan)
(59703, nan)
(59704, nan)
(59705, nan)
(59706, nan)
(59707, nan)
(59708, nan)
(59709, nan)
(59710, nan)
(59711, nan)
(59712, nan)
(59713, nan)
(59714, nan)
(59715, nan)
(59716, nan)
(59717, nan)
(59718, nan)
(59719, nan)
(59720, nan)
(59721, nan)
(59722, nan)
(59723, nan)
(59724, nan)
(59725, nan)
(59726, nan)
(59727, nan)
(59728, nan)
(59729, nan)
(59730, nan)
(59731, nan)
(59732, nan)
(59733, nan)
(59734, nan)
(59735, nan)
(59736, nan)
(59737, nan)
(59738, nan)
(59739, nan)
(59740, nan)
(59741, nan)
(59742, nan)
(59743, nan)
(59744, nan)
(59745, nan)
(59746, nan)
(59747, nan)
(59748, nan)
(59749, nan)
(59750, nan)
(59751, nan)
(59752, nan)
(59753, nan)
(59754, nan)
(59755, nan)
(59756, nan)
(59757, nan)
(59758, nan)
(59759, nan)
(59760, nan)
(59761, nan)
(59762, nan)
(59763, nan)
(59764, nan)
(59765, nan)
(59766, nan)
(59767, nan)
(59768, nan)
(59769, nan)
(59770, nan)
(59771, nan)
(59772, nan)
(59773, nan)
(59774, nan)
(59775, nan)
(59776, nan)
(59777, nan)

In [278]:
sorted_news_items = sorted(dated_news_item, key = lambda x: x[0])

In [295]:
len(sorted_news_items)

134717

In [301]:
begin_date = datetime.date(2017,1,1)
news_items_2017 = [news_item for news_item in sorted_news_items if news_item[0] >=  begin_date]

In [383]:
def date2unixtime(date):
    unixtime = time.mktime(date.timetuple())
    return unixtime

def date_conversion_distinctive_timestamp(date_seq):
    # find how many documents in a specific date
    date_cnt = Counter(date_seq)
    ordered_dates = sorted(date_cnt.keys())
    timestamps = []
    for date in ordered_dates:
        date_in_unixtime = date2unixtime(date)
        next_date = date + datetime.timedelta(days=1)
        doc_num_in_day = date_cnt[date]
        next_day_in_unixtime = date2unixtime(next_date)
        #time_interval = (next_day_in_unixtime - date_in_unixtime) / doc_num_in_day
        #daily_timestamps = np.arange(date_in_unixtime, next_day_in_unixtime, time_interval)
        daily_timestamps = np.linspace(date_in_unixtime, next_day_in_unixtime, num=doc_num_in_day, endpoint=False)
        daily_timestamps = daily_timestamps.tolist()
        assert daily_timestamps[-1] < next_day_in_unixtime
        timestamps.extend(daily_timestamps)
        '''
        print('date', date)
        print('next_date', next_date)
        print('doc_num_in_day', doc_num_in_day)
        print('date_in_unixtime',date_in_unixtime)
        print('next_day_in_unixtime',next_day_in_unixtime)
        print((next_day_in_unixtime - date_in_unixtime))
        print('time_interval',time_interval)
        print('\n')
        '''
    return timestamps

# change date into distinctive timestamps
date_seq = [news_item[0] for news_item in news_items_2017]
timestamps = date_conversion_distinctive_timestamp(date_seq)

In [385]:
for i, _ in enumerate(timestamps):
    if i == 0: continue
    assert timestamps[i] > timestamps[i-1], (i, timestamps[i], timestamps[i-1])

In [395]:
date, word_distribution, word_count = zip(*news_items_2017)
idx = range(len(news_items_2017))
news_items_2017 = zip(idx, timestamps, word_distribution, word_count)

In [398]:
import json

with open('./data/all_the_news_2017.json', 'w') as f:
    json.dump(news_items_2017, f)

In [400]:
print('vocabulary size %d' % len(id2word))

vocabulary size 56720
